In [46]:
# ─────────────────────────────────────────────────────────────
# Cell 1 ▸ Imports for ANN training
# ─────────────────────────────────────────────────────────────
import joblib
from keras import callbacks, layers, models
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED);
tf.random.set_seed(SEED)

In [47]:
# ─────────────────────────────────────────────────────────────
#  Cell 2 ▸ Load & tidy the data
# ─────────────────────────────────────────────────────────────
csv_path = "data/processed/preprocessed_dataset.csv"
raw = pd.read_csv(csv_path)

# Harmonise column names
rename_map = {
    "Time_ms": "time",
    "Pc_bar": "chamb_pressure",
    "Tc_K": "cham_temp",
    "Pinj_bar": "injection_pres",
    "rho_kgm3": "density",
    "mu_Pas": "viscosity",
    "angle_shadow_deg": "angle_shadow",
    "len_shadow_L_D": "length_shadow",
    "angle_mie_deg": "angle_mie",
    "len_mie_L_D": "length_mie",
}
df = raw.rename(columns=rename_map)

INPUTS = ["time", "chamb_pressure", "cham_temp", "injection_pres", "density", "viscosity"]
TARGETS = ["angle_mie", "length_mie", "angle_shadow", "length_shadow"]

print("Data shape  :", df.shape)
display(df.head())

Data shape  : (726, 11)


,run,time,chamb_pressure,cham_temp,injection_pres,density,viscosity,angle_shadow,length_shadow,angle_mie,length_mie
0,ETH-01,0.000,55.0318,192.0295,98.8645,810.7202,0.0019,16.6945,13.1266,12.9373,17.5713
1,ETH-01,0.025,55.0057,192.0158,98.8741,810.7183,0.0019,16.6945,13.1266,12.9373,17.5713
2,ETH-01,0.050,55.0081,191.9882,98.9074,810.7184,0.0019,16.6945,13.1266,12.9373,17.5713
3,ETH-01,0.075,55.0164,192.0820,98.8550,810.7191,0.0019,16.6945,13.1266,12.9373,17.5713
4,ETH-01,0.100,55.0125,191.9880,98.8788,810.7188,0.0019,16.6945,20.2047,12.9373,24.5066


In [48]:
# ─────────────────────────────────────────────────────────────
#  Cell 3 ▸ Train-test split  (stratified by experimental run)
# ─────────────────────────────────────────────────────────────
X, y = df[INPUTS], df[TARGETS]
runs = df["run"]  # stratification label
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.20, random_state=42, stratify=runs)

print(f"Train: {X_tr.shape}   Test: {X_te.shape}")

# Scale context for unitless metrics
TARGET_RANGE = (y.max() - y.min()).to_dict()
TARGET_STD   = (y.std(ddof=0)).to_dict()

Train: (580, 6)   Test: (146, 6)


In [49]:
# ─────────────────────────────────────────────────────────────
# Cell 4 ▸ Prepare scaled train and test data (reuse X_tr, y_tr, X_te, y_te)
# ─────────────────────────────────────────────────────────────
# Use the same train-test split as ML baselines (reuse your existing X_tr, y_tr, X_te, y_te)

# Scale input features
input_scaler = StandardScaler()
X_train_scaled = input_scaler.fit_transform(X_tr)
X_test_scaled = input_scaler.transform(X_te)

# Scale output targets
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_tr)
y_test_scaled = target_scaler.transform(y_te)

# Save scalers
joblib.dump(input_scaler, "models/ann_input_scaler.joblib")
joblib.dump(target_scaler, "models/ann_target_scaler.joblib")

print("Scaling completed. Shapes:", X_train_scaled.shape, y_train_scaled.shape)

Scaling completed. Shapes: (580, 6) (580, 4)


In [50]:
# ─────────────────────────────────────────────────────────────
# Cell 5 ▸ Build the ANN model architecture
# ─────────────────────────────────────────────────────────────
def build_deep_ann(input_dim, output_dim):
    model = models.Sequential(
        [
            layers.InputLayer(input_shape=(input_dim,)),
            layers.Dense(256, activation="relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Dense(128, activation="relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(64, activation="relu"),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(output_dim, activation="linear"),
        ]
    )

    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


model = build_deep_ann(X_train_scaled.shape[1], y_train_scaled.shape[1])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 256)            │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,996 (175.77 KB)

 Trainable params: 44,100 (172.27 KB)

 Non-trainable params: 896 (3.50 KB)

In [51]:
# ─────────────────────────────────────────────────────────────
# Cell 6 ▸ Train the ANN model with EarlyStopping
# ─────────────────────────────────────────────────────────────
early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=30, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10)

history = model.fit(
    X_train_scaled,
    y_train_scaled,
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose="2",
)

Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [56]:
# Configure TensorFlow for deterministic operations
tf.config.experimental.enable_op_determinism()

In [59]:
# ─────────────────────────────────────────────────────────────
# Cell 7 ▸ Evaluate the ANN model on the test set
# ─────────────────────────────────────────────────────────────

# Inverse transform predictions & evaluate on original scale
y_pred_scaled = model.predict(X_test_scaled, verbose=0)
y_pred = target_scaler.inverse_transform(y_pred_scaled)

r2_scores, mae_scores, mse_scores, rmse_scores = [], [], [], []
nmae_range, nrmse_range, nmae_std, nrmse_std = [], [], [], []

for i, col in enumerate(y_te.columns):
    y_true_col = y_te[col].to_numpy()
    y_pred_col = y_pred[:, i]
    r2  = r2_score(y_true_col, y_pred_col)
    mae = mean_absolute_error(y_true_col, y_pred_col)
    mse = mean_squared_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mse)

    r2_scores.append(r2); mae_scores.append(mae); mse_scores.append(mse); rmse_scores.append(rmse)
    nmae_range.append(mae / TARGET_RANGE[col])
    nrmse_range.append(rmse / TARGET_RANGE[col])
    nmae_std.append(mae / TARGET_STD[col])
    nrmse_std.append(rmse / TARGET_STD[col])

    print(f"{col:>15}: R²={r2:.4f}, MAE={mae:.4f}, RMSE={rmse:.4f}, "
          f"NMAE(range)={mae/TARGET_RANGE[col]:.4f}, NRMSE(range)={rmse/TARGET_RANGE[col]:.4f}")

print(f"\nOverall mean R²: {np.mean(r2_scores):.4f}")
print(f"Overall mean MAE: {np.mean(mae_scores):.4f}  RMSE: {np.mean(rmse_scores):.4f}")
print(f"Overall NMAE(range): {np.mean(nmae_range):.4f}  NRMSE(range): {np.mean(nrmse_range):.4f}")

# Save model & predictions (original scale)
model.save("models/ANN_improved_regressor.h5")
pd.DataFrame(y_pred, columns=y_te.columns).to_csv("outputs/ANN_improved_predictions.csv", index=False)
y_te.to_csv("outputs/ANN_improved_actuals.csv", index=False)

# Save per-target metrics with normalized columns
ann_metrics = pd.DataFrame(
    {
        "model": ["ANN"] * len(y_te.columns),
        "target": list(y_te.columns),
        "r2": r2_scores,
        "mae": mae_scores,
        "mse": mse_scores,
        "rmse": rmse_scores,
        "nmae_range": nmae_range,
        "nrmse_range": nrmse_range,
        "nmae_std": nmae_std,
        "nrmse_std": nrmse_std,
    }
)
ann_metrics.to_csv("outputs/ANN_improved_metrics.csv", index=False)
print("Saved improved model and predictions.")

      angle_mie: R²=0.9937, MAE=0.0783, RMSE=0.0995, NMAE(range)=0.0175, NRMSE(range)=0.0222
     length_mie: R²=0.9927, MAE=3.2157, RMSE=4.4444, NMAE(range)=0.0165, NRMSE(range)=0.0228
   angle_shadow: R²=0.9849, MAE=0.1214, RMSE=0.1466, NMAE(range)=0.0254, NRMSE(range)=0.0307
  length_shadow: R²=0.9929, MAE=3.3191, RMSE=4.5547, NMAE(range)=0.0165, NRMSE(range)=0.0227

Overall mean R²: 0.9911
Overall mean MAE: 1.6836  RMSE: 2.3113
Overall NMAE(range): 0.0190  NRMSE(range): 0.0246
Saved improved model and predictions.
